In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
import torch.optim as optim
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

下载并加载数据集

In [4]:
train_data = torchvision.datasets.MNIST(
    root = './datasets/MNIST',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True,
)

train_loader = Data.DataLoader(
    dataset = train_data,
    batch_size = 128,
    shuffle = True,
    num_workers = 1,
)

test_data = torchvision.datasets.MNIST(
    root = './datasets/MNIST',
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = False,
)
test_data_x = test_data.data.type(torch.FloatTensor) / 255.0
test_data_x = torch.unsqueeze(test_data_x, dim = 1)
test_data_y = test_data.targets
print(test_data_y.shape)

torch.Size([10000])


查看数据集

In [5]:
for epoch, (data, target) in enumerate(train_loader):
    if epoch > 0:
        break
    print(data.shape)
    print(target.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128])


定义网络

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #定义第一个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 1,
                      out_channels = 16,
                      kernel_size = 3,
                      stride = 1,
                      padding = 1,
            ),
            nn.ReLU(),
            nn.AvgPool2d(
                kernel_size = 2,
                stride = 2,
            ),
        )
        #定义第二个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        #定义第第一个全连接层
        self.fc = nn.Sequential(
            nn.Linear(
                in_features = 32*7*7,
                out_features = 128,
            ),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        self.out = nn.Linear(64, 10)
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = self.out(x)
        return output

In [7]:
net = ConvNet()
print(net)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1568, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (out): Linear(in_features=64, out_features=10, bias=True)
)


PyTorchViz库可视化网络结构

In [8]:
from torchviz import make_dot
x = torch.randn(1, 1, 28, 28).requires_grad_(True)
y = net(x)
MyConvnetvis = make_dot(y, params = dict(list(net.named_parameters()) + [('x', x)]))
MyConvnetvis.format = 'png'
MyConvnetvis.directory = './nets/charpt4/net_vis'
MyConvnetvis.view()

ModuleNotFoundError: No module named 'torchviz'

定义优化器

In [9]:
optimizer = optim.Adam(net.parameters(), lr = 0.00003)
criteon = nn.CrossEntropyLoss()
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        output = net(data)
        loss = criteon(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step
        if batch_idx%50 == 0:
            print('Train Epoch:{}[{}/{} ({:.0f}%)]\tloss:{:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset),
                                                                     100.*batch_idx/len(train_loader), loss.item()))
        

C:\developer\Anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch:0[0/60000 (0%)]	loss:2.304132
Train Epoch:0[6400/60000 (11%)]	loss:2.307573
Train Epoch:0[12800/60000 (21%)]	loss:2.311388
Train Epoch:0[19200/60000 (32%)]	loss:2.307529
Train Epoch:0[25600/60000 (43%)]	loss:2.307341
Train Epoch:0[32000/60000 (53%)]	loss:2.308856
Train Epoch:0[38400/60000 (64%)]	loss:2.308000
Train Epoch:0[44800/60000 (75%)]	loss:2.310258
Train Epoch:0[51200/60000 (85%)]	loss:2.301616
Train Epoch:0[57600/60000 (96%)]	loss:2.306718
Train Epoch:1[0/60000 (0%)]	loss:2.297678
Train Epoch:1[6400/60000 (11%)]	loss:2.310760
Train Epoch:1[12800/60000 (21%)]	loss:2.306643
Train Epoch:1[19200/60000 (32%)]	loss:2.297122
Train Epoch:1[25600/60000 (43%)]	loss:2.308167
Train Epoch:1[32000/60000 (53%)]	loss:2.302778
Train Epoch:1[38400/60000 (64%)]	loss:2.304413
Train Epoch:1[44800/60000 (75%)]	loss:2.311292
Train Epoch:1[51200/60000 (85%)]	loss:2.308395
Train Epoch:1[57600/60000 (96%)]	loss:2.298121
Train Epoch:2[0/60000 (0%)]	loss:2.309751
Train Epoch:2[6400/60000 (11%)

使用tensorboardX进行可视化

In [11]:
from tensorboardX import SummaryWriter
SumWriter = SummaryWriter(log_dir = './vis/charpt4/log')
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0003)
criteon = nn.CrossEntropyLoss()
train_loss = 0
for epoch in range(5):
    for step, (data, target) in enumerate(train_loader):
        output = net(data)
        loss = criteon(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss + train_loss
        #计算迭代次数
        niter = epoch * len(train_loader) + step +1
        if niter % 100 == 0:
            SumWriter.add_scalar('train_loss', train_loss.item()/niter, global_step = niter)
            output = net(test_data_x)
            _,pre_lab = torch.max(output, 1)
            acc = accuracy_score(test_data_y, pre_lab)
            SumWriter.add_scalar('test_acc', acc.item(), niter)
            b_x_im = vutils.make_grid(data, nrow = 12)
            SumWriter.add_image('train_image sample', b_x_im, niter)
            for name, param in net.named_parameters():
                SumWriter.add_histogram(name, param.data.numpy(), niter)

使用visdom可视化

In [13]:
from visdom import Visdom
from sklearn.datasets import load_iris
iris_x, iris_y = load_iris(return_X_y = True)

In [14]:
vis = Visdom()
vis.scatter(iris_x[:,0:2], Y = iris_y+1, win= 'windows1', env = 'main')
vis.scatter(iris_x[:,0:3], Y = iris_y+1, win='3D 散点图', env='main', opts = dict(marersize = 4, xlabel = '特征1', ylabel = '特征2'))


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\developer\Anaconda\envs\pytorch\lib\site-pack

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\developer\Anaconda\envs\pytorch\lib\site-pack

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\developer\Anaconda\envs\pytorch\lib\site-packages\urllib3\connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\developer\Anaconda\envs\pytorch\lib\site-pack

False

In [24]:
import numpy as np
x = torch.linspace(-6, 6, 100).view((-1, 1))
sigmoid = torch.nn.Sigmoid()
sigmoidy = sigmoid(x)
ploty = sigmoidy
plotx = x
vis.line(Y = ploty, X= plotx, win = 'line plot', env ='main', opts = dict(dash = np.array(['solid']), legend = ['Sigmoid']))

AssertionError: X and Y should be the same shape